In [1]:
import pandas as pd
from ib_insync import *
util.startLoop() #only for jupyter IDEs

In [2]:
ib = IB()

In [3]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [4]:
ib.positions()

[]

In [5]:
contract = Forex('EURUSD')
contract

Forex('EURUSD', exchange='IDEALPRO')

In [6]:
ib.qualifyContracts(contract)
#to make sure the contract is unambiguous

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

In [7]:
data = ib.reqMktData(contract) #subscibe to market data
data

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'))

In [8]:
data.bid

nan

In [9]:
ib.cancelMktData #unsubscribe from market data

<bound method IB.cancelMktData of <IB connected to 127.0.0.1:7497 clientId=1>>

In [10]:
ib.reqMktData(contract)

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'))

In [11]:
ticker = ib.ticker(contract)

In [12]:
ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'))

In [13]:
ticker.bid

nan

# Streaming Tick data

In [14]:
ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'))

In [15]:
def onPendingTickers(tickers): #what will happen after receiving a new tick
    print('time:{} | Bid:{} | Ask:{}'.format(ticker.time, ticker.bid, ticker.ask), end = '\n')

In [16]:
ib.pendingTickersEvent+= onPendingTickers   #activates the function 

In [17]:
ib.pendingTickersEvent-= onPendingTickers   #deactivates the function

# Streaming data for Multiple tickers

In [18]:
from IPython.display import display, clear_output

In [19]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts) #checking if all the contracts are unambiguous

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'),
 Forex('USDJPY', conId=15016059, exchange='IDEALPRO', localSymbol='USD.JPY', tradingClass='USD.JPY'),
 Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD'),
 Forex('USDCAD', conId=15016062, exchange='IDEALPRO', localSymbol='USD.CAD', tradingClass='USD.CAD'),
 Forex('AUDUSD', conId=14433401, exchange='IDEALPRO', localSymbol='AUD.USD', tradingClass='AUD.USD')]

In [20]:
for c in contracts:
    ib.reqMktData(c)

In [22]:
df = pd.DataFrame(index= [c.pair() for c in contracts],
                 columns= ['bidsize', 'bid', 'ask', 'asksize', 'high', 'low', 'close'])
df

,bidsize,bid,ask,asksize,high,low,close
EURUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDJPY,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBPUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDCAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AUDUSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
def onPendinggticker(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
        t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait = True)
    display(df)

In [29]:
ib.pendingTickersEvent+= onPendinggticker #activates the function

,bidsize,bid,ask,asksize,high,low,close
EURUSD,1000000.0,1.0905,1.09052,2000000.0,1.09195,1.08845,1.0907
USDJPY,2500000.0,131.401,131.407,1500000.0,131.575,130.775,131.32
GBPUSD,3000000.0,1.24626,1.24631,1000000.0,1.2487,1.2437,1.2462
USDCAD,500000.0,1.34741,1.34744,3000000.0,1.34895,1.3447,1.3457
AUDUSD,500000.0,0.66878,0.6688,1000000.0,0.67255,0.66845,0.67215


In [30]:
ib.pendingTickersEvent -= onPendinggticker # deactivates the function

In [31]:
for c in contracts:
    ib.cancelMktData(c)

In [32]:
ib.disconnect()